In [12]:
from __future__ import print_function
from os.path import isfile, join

import segyio
from os import listdir
import numpy as np
np.set_printoptions(threshold=np.nan) #Print todo el numpy.array
import scipy.misc

In [3]:

def readSEGY(filename):
    print('Loading data cube from',filename,'with:')

    # Read full data cube
    data = segyio.tools.cube(filename)

    # Put temporal axis first
    data = np.moveaxis(data, -1, 0)

    #Make data cube fast to acess
    data = np.ascontiguousarray(data,'float32')

    #Read meta data
    segyfile = segyio.open(filename, "r")
    print('  Crosslines: ', segyfile.xlines[0], ':', segyfile.xlines[-1])
    print('  Inlines:    ', segyfile.ilines[0], ':', segyfile.ilines[-1])
    print('  Timeslices: ', '1', ':', data.shape[0])

    #Make dict with cube-info
    data_info = {}
    data_info['crossline_start'] = segyfile.xlines[0]
    data_info['inline_start'] = segyfile.ilines[0]
    data_info['timeslice_start'] = 1 #Todo: read this from segy
    data_info['shape'] = data.shape
    #Read dt and other params needed to do create a new


    return data, data_info


In [6]:
# Writes out_cube to a segy-file (out_filename) with same header/size as in_filename
def writeSEGY(out_filename, in_filename, out_cube):
    #Select last channel
    if type(out_cube) is list:
        out_cube = out_cube[-1]

    print('Writing interpretation to ' + out_filename)
    #Copy segy file
    from shutil import copyfile
    copyfile(in_filename, out_filename)

    # Moving temporal axis back again
    out_cube = np.moveaxis(out_cube, 0,-1)

    #Open out-file
    with segyio.open(out_filename, "r+") as src:
        iline_start = src.ilines[0]
        dtype = src.iline[iline_start].dtype
        # loop through inlines and insert output
        for i in src.ilines:
            iline = out_cube[i-iline_start,:,:]
            src.iline[i] = np.ascontiguousarray(iline.astype(dtype))

    # Moving temporal axis first again - just in case the user want to keep working on it
    out_cube = np.moveaxis(out_cube, -1, 0)

    print('Writing interpretation - Finished')
    return

In [ ]:
readSEGY('NS2600-3000_2700-3100.sgy')
print

In [19]:
with segyio.open('FS2600-3000_2700-3100.sgy','r') as fs:
    print(fs.ilines)
    print(fs.iline[2600])
    print(fs.depth_slice[700])
    

[2600 2601 2602 2603 2604 2605 2606 2607 2608 2609 2610 2611 2612 2613
 2614 2615 2616 2617 2618 2619 2620 2621 2622 2623 2624 2625 2626 2627
 2628 2629 2630 2631 2632 2633 2634 2635 2636 2637 2638 2639 2640 2641
 2642 2643 2644 2645 2646 2647 2648 2649 2650 2651 2652 2653 2654 2655
 2656 2657 2658 2659 2660 2661 2662 2663 2664 2665 2666 2667 2668 2669
 2670 2671 2672 2673 2674 2675 2676 2677 2678 2679 2680 2681 2682 2683
 2684 2685 2686 2687 2688 2689 2690 2691 2692 2693 2694 2695 2696 2697
 2698 2699 2700]


AttributeError: 'numpy.ndarray' object has no attribute 'depth_slice'